<a href="https://colab.research.google.com/github/ShotaroBaba/Machine_Learning_Practice/blob/master/image_prediction_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes


In [1]:
import torch
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import os


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# Check if a device is cuda or not.
print(device)

cpu


In [3]:
!kaggle datasets download -d alessiocorrado99/animals10

animals10.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
!ls

animals10.zip  raw-img	sample_data  translate.py


In [5]:
!if [ ! -d raw-img ]; then unzip animals10.zip &> /dev/null ; fi

In [6]:
!ls raw-img

cane	 elefante  gallina  mucca   ragno
cavallo  farfalla  gatto    pecora  scoiattolo


In [7]:
# Make labels using file names.

animal_to_number={}
number_to_animal={}
i=0

for root,dirs,files in os.walk(u"raw-img"):
  for dir in dirs:
    print(dir)
    animal_to_number[dir]=i
    number_to_animal[i]=dir
    i+=1

print(animal_to_number)
print(number_to_animal)




cavallo
pecora
cane
gallina
gatto
mucca
scoiattolo
ragno
elefante
farfalla
{'cavallo': 0, 'pecora': 1, 'cane': 2, 'gallina': 3, 'gatto': 4, 'mucca': 5, 'scoiattolo': 6, 'ragno': 7, 'elefante': 8, 'farfalla': 9}
{0: 'cavallo', 1: 'pecora', 2: 'cane', 3: 'gallina', 4: 'gatto', 5: 'mucca', 6: 'scoiattolo', 7: 'ragno', 8: 'elefante', 9: 'farfalla'}


In [8]:
# Take all images into ram.

list_of_files={}

# First, create a list of image file path.
for root,dirs,files in os.walk(u"raw-img"):
    for dir in dirs:
      index_num=animal_to_number[str(dir)]
      tmp_list=[]
      for root_,dirs_,files_ in os.walk(os.path.join(u"raw-img",dir)):
          for file_ in files_:
            tmp_list.append(os.path.join(os.path.join(u"raw-img",dir),file_))
      
      list_of_files[index_num]=tmp_list[:400]


In [9]:
# Load all files into a list using pillow
from PIL import Image, ImageOps
import random
import numpy as np
# Check if a retrieved image is correct.

sample_image=random.sample(range(0,len(list_of_files[9])),k=1)

img=Image.open(list_of_files[9][sample_image[0]])

print(list_of_files[9][sample_image[0]])

len(sum(list_of_files.values(),[]))

raw-img/farfalla/ea36b60b21f5033ed1584d05fb1d4e9fe777ead218ac104497f5c97faee9bdba_640.jpg


4000

In [10]:
# Now converting all images into numpy array.

size=128,128

# Create image (array) and label data.
total_data_num=len(sum(list_of_files.values(),[]))
x_data=np.empty((total_data_num,128,128))
y_data=np.empty((total_data_num))

k=0
for i in list_of_files.keys():
  for j in range(0, len(list_of_files[i])):
    y_data[k]=i
    if k%2000==0:
      print("Currently {} images are loaded".format(k))
    
    # Read image, resize and convert it into a numpy array.
    x_data[k]=np.array(Image.open(list_of_files[i][j]).convert('L').resize(size))
    k+=1

print("In total, {} images have been loaded.".format(k))


Currently 0 images are loaded
Currently 2000 images are loaded
In total, 4000 images have been loaded.


In [11]:
# x_data /= 255
x_data.astype('float64')
y_data.astype(int)
x_data=np.reshape(x_data,(k,1,128,128))
y_data=np.reshape(y_data,(k,1)).astype(int)

In [12]:
print(x_data.shape)
print(y_data.shape)

(4000, 1, 128, 128)
(4000, 1)


In [13]:
train_x,test_x,train_y,test_y=train_test_split(x_data,y_data,test_size=0.24,random_state=100)
train_x=torch.from_numpy(train_x)
train_y=torch.from_numpy(train_y)
test_x=torch.from_numpy(test_x)
test_y=torch.from_numpy(test_y)

print(train_x)
print(train_y)
print(test_x)
print(test_y)


tensor([[[[ 82.,  61.,  55.,  ..., 252., 208., 157.],
          [ 91.,  69.,  59.,  ..., 194., 116., 157.],
          [ 75.,  63.,  68.,  ..., 152.,  62.,  98.],
          ...,
          [ 67.,  64.,  77.,  ...,  86.,  90.,  84.],
          [ 79.,  66.,  73.,  ..., 102., 105.,  82.],
          [ 88.,  70.,  77.,  ..., 137.,  91.,  82.]]],


        [[[ 73.,  74., 104.,  ..., 170., 170., 166.],
          [ 83.,  92., 135.,  ..., 160., 154., 164.],
          [ 91.,  93., 141.,  ..., 172., 162., 175.],
          ...,
          [174., 143., 156.,  ..., 191., 169., 181.],
          [145., 147., 158.,  ..., 176., 164., 186.],
          [144., 147., 137.,  ..., 172., 187., 188.]]],


        [[[ 81., 100.,  88.,  ..., 110., 112., 117.],
          [ 90.,  97., 101.,  ..., 112., 119., 120.],
          [ 82.,  80.,  95.,  ..., 112., 111., 120.],
          ...,
          [182., 122., 130.,  ...,  67.,  74.,  83.],
          [137.,  91., 107.,  ...,  59.,  74.,  90.],
          [155., 116., 157., 

In [17]:
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout,MSELoss
from torch.optim import Adam, SGD

class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
        )

        self.linear_layers = Sequential(
            Linear(16 * 16 * 16, 10)
        )


    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

model = Net()

optimizer = Adam(model.parameters(), lr=0.07)

criterion = MSELoss()

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=4096, out_features=10, bias=True)
  )
)


In [18]:
def train(epoch):
    model.train()
    tr_loss = 0
    # getting the training set
    x_train, y_train = Variable(train_x), Variable(train_y)
    # getting the validation set
    x_test, y_test = Variable(test_x), Variable(test_y)
    # converting the data into GPU format
    print(torch.cuda.is_available())
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_test = x_test.cuda()
        y_test = y_test.cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    x_train=x_train.float()
    x_test=x_test.float()
    print(x_test.shape)
    print(x_train.shape)
    output_train = model(x_train)
    output_val = model(x_test)

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train)
    loss_val = criterion(output_val, y_test)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

In [19]:
# defining the number of epochs
n_epochs = 10
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
for epoch in range(n_epochs):
    train(epoch)

False
torch.Size([960, 1, 128, 128])
torch.Size([3040, 1, 128, 128])


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([3040, 1])) that is different to the input size (torch.Size([3040, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([960, 1])) that is different to the input size (torch.Size([960, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


RuntimeError: ignored